# Programa para Calcular Faturamento com Pandas

In [115]:
import pandas as pd
from glob import glob
import locale
locale.setlocale(locale.LC_ALL, 'pt_BR.UTF-8')

'pt_BR.UTF-8'

No Codigo abaixo, estamos usando a biblioteca glob para juntar todos arquivos de vendas em apenas um, fazemos o mesmo com os arquivos de devolução

In [116]:
Vendas = sorted(glob(r"Vendas-Devolucoes/Vendas/Vendas - *.csv"))
Devolucoes = sorted(glob(r"Vendas-Devolucoes/Devolucoes/Devolucoes - *.csv"))

Na função abaixo, usamos o comando "global" para fazer as variaveis ao lado possa ser utilizado em todo canto do codigo e não seja apenas uma variavel de escopo

In [117]:
def todos_arquivos():
    global todos_arquivos_venda, todos_arquivos_devolucao
    todos_arquivos_venda = pd.concat((pd.read_csv(cont) for cont in Vendas), ignore_index= True)  
    todos_arquivos_devolucao = pd.concat((pd.read_csv(cont) for cont in Devolucoes), ignore_index=True)
    todos_arquivos_devolucao = todos_arquivos_devolucao.dropna(how="any", axis= 1)
    todos_arquivos_venda = todos_arquivos_venda.dropna(how="any", axis= 1)

Na função abaixo, fazemos os calculos multiplicando o valor unitário do produto pela quantidade vendida, assim criando uma coluna apenas com esses valores. Fazemos isso tanto com os valores de devolução, tanto com o de vendas

In [118]:
def calculo_total_vendido_devolucao(Valor_Total_Vendido, Valor_Total_Devolvido, Preco_Unitario, Preço_Unitário, Quantidade_Devolvida, Quantidade_Vendida):
    todos_arquivos_venda[Valor_Total_Vendido]= todos_arquivos_venda[Preco_Unitario] * todos_arquivos_venda[Quantidade_Vendida]
    todos_arquivos_devolucao[Valor_Total_Devolvido]= todos_arquivos_devolucao[Preço_Unitário] * todos_arquivos_devolucao[Quantidade_Devolvida]

Na função abaixo, chamamos a funçao "calculo_total_vendido_devolucao". Além disso, pegamos a tabela feita na funçao acima e usamos o comando "groupby". O comando "groupby" organiza todos os elementos da coluna indicada, e o comando após o "groupby" serve para as demais colunas da tabela, nesse caso ele soma as demais colunas. Após organizar as tabelas, fazemos os calculos de faturamento subtraindo, e logo depois transformando as tuplas em um dicionario para trabalhar melhor com os valores, assim pegamos o maior valor em faturamento, transfomarmos para a moeda real do Brasil, e pegamos o nome do produto com maior faturamento 

In [119]:
def maior_faturamento_produto():
    calculo_total_vendido_devolucao("Valor Total Vendido", "Valor Total Devolvido", "Preco Unitario", "Preço Unitário", "Quantidade Devolvida", "Quantidade Vendida")
    produto_venda = todos_arquivos_venda.groupby(["Produto"]).sum()
    produto_devolucao = todos_arquivos_devolucao.groupby(["Produto"]).sum()
    subtracao_faturamento = abs(produto_devolucao["Valor Total Devolvido"] - produto_venda["Valor Total Vendido"])
    dic = dict(subtracao_faturamento)
    maior_numero = max(dic.values())
    formatacao_em_real = locale.currency(maior_numero, grouping=True) 
    for chave, valor in dic.items():
        if valor == maior_numero:
            return f"o produto {chave} teve o maior faturamento comparado a outros produtos, sendo seu faturamento de {formatacao_em_real}"

Nessa função, fazemos quase o mesmo que na função acima, porem dessa vez achamos o valor e o nome do produto q teve maior devolução 

In [120]:
def produto_maior_devolucao():
    numero_devolucao = todos_arquivos_devolucao.groupby(["Produto"]).max()
    dicionario_numeros_devolucoes = dict(numero_devolucao["Valor Total Devolvido"])
    maior_numero_devolucao = max(dicionario_numeros_devolucoes.values())
    formatacao_em_real_devolucao = locale.currency(maior_numero_devolucao, grouping=True)
    return f"o valor com maior numero de devoluções foi {max(dicionario_numeros_devolucoes)} com {formatacao_em_real_devolucao}"

Abaixo apenas chamamos as funções para elas rodarem

In [125]:
maior_faturamento_produto()

'o produto Televisão teve o maior faturamento comparado a outros produtos, sendo seu faturamento de R$ 14.460.000,00'

In [124]:
produto_maior_devolucao()

'o valor com maior numero de devoluções foi Televisão com R$ 26.500,00'